In [1]:
# just pip install all necesay libraries and comment out Colab specific lines when running on a computer

import os


# comment out following section for PC
!pip install chess
import shutil
if os.path.exists('/content/ChessNN'):
  shutil.rmtree('/content/ChessNN')

!git clone https://github.com/AlphaCentaureye/ChessNN.git

import sys
sys.path.insert(0, '/content/ChessNN/src/')



# always keep
import tensorflow as tf
import chess
from environment import Board
from agent import Agent
from learning_algs import Q_learn
import numpy as np




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.8 MB/s eta 0:00:00
Cloning into 'ChessNN'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 309 (delta 35), reused 34 (delta 16), pack-reused 255
Receiving objects: 100% (309/309), 62.66 KiB | 8.95 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [ ]:
# train network

environment = Board()
agent = Agent(verbose=1)
agent.init_network()
#print(agent.model.summary())

learn = Q_learn(agent, environment)
learn.learn(iterations=30, updateThreshold=1, explorationRateRatio=5, display=True)

try:
    agent.saveNN() # optional path param, but defaults to path /content/savedNNs/
except Exception as e:
    print(e)


In [ ]:
from logging import exception
from google.colab import files
try:
  files.download('/content/savedNNs/chessNN_model.zip')
except exception as e:
  print(e)

In [ ]:
# play against network

board_env = Board()

while board_env.board.result() == '*':
    prediction_vector = agent.find_move(Agent.one_hot_encode(board_env.board, color=chess.WHITE))
    move = Agent.one_hot_decode(prediction_vector, board_env.board)
    board_env.play(move)
    if not(board_env.board.result() == '*'):
        break
    while True:
        move_input = input('enter move (ex: a2a3): ')
        move = chess.Move.from_uci(move_input)
        if move in board_env.board.legal_moves:
            board_env.play(move)
            break
        else:
            print('** invalid move **')

print('GAME OVER')
print('winner: ', 'white' if board_env.board.result() == '1-0' else ('black' if board_env.board.result() == '0-1' else 'draw'))
